In [37]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config
import mysql.connector

In [11]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password)

In [6]:
cursor = cnx.cursor()

## Create "music" database to dump Spotify information into

In [36]:
stmt = """CREATE DATABASE music DEFAULT CHARACTER SET 'utf8'"""
cursor.execute(stmt)

InternalError: Unread result found

## Connect to "music" database

In [137]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'music')

In [138]:
cursor = cnx.cursor()

In [63]:
tables = {}
tables['artists'] = (
    "CREATE TABLE artists ("
    "  artist_id varchar(50) NOT NULL,"
    "  name varchar(100) NOT NULL,"
    "  genre varchar(100) NOT NULL,"
    "  popularity int(3),"
    "  PRIMARY KEY (artist_id)"
    ") ENGINE=InnoDB")

In [64]:
tables['albums'] = (
    "CREATE TABLE albums ("
    "  album_id varchar(50) NOT NULL,"
    "  album_name varchar(100) NOT NULL,"
    "  artist_id1 varchar(50) NOT NULL,"
    "  artist_id2 varchar(50),"
    "  artist_id3 varchar(50),"
    "  release_date DATE,"
    "  total_tracks int(2),"
    "  PRIMARY KEY (album_id)"
    ") ENGINE=InnoDB")

In [65]:
tables['songs'] = (
    "CREATE TABLE songs ("
    "  song_id varchar(50) NOT NULL,"
    "  song_name varchar(100) NOT NULL,"
    "  artist_id1 varchar(50) NOT NULL,"
    "  artist_id2 varchar(50),"
    "  artist_id3 varchar(50),"
    "  album_id varchar(50),"
    "  track_number int(2),"
    "  popularity int(2),"
    "  PRIMARY KEY (song_id)"
    ") ENGINE=InnoDB")

In [66]:
tables['features'] = (
    "CREATE TABLE features ("
    "  song_id varchar(50) NOT NULL,"
    "  feature_id varchar(100) NOT NULL,"
    "  danceability float,"
    "  energy float,"
    "  key_ float,"
    "  loudness float,"
    "  mode float,"
    "  speechiness float,"
    "  acousticness float,"
    "  instrumentalness float,"
    "  liveness float,"
    "  valence float,"
    "  tempo float,"
    "  duration_ms float,"
    "  time_signature float,"
    "  PRIMARY KEY (feature_id)"
    ") ENGINE=InnoDB")

In [67]:
tables['sections'] = (
    "CREATE TABLE sections ("
    "  song_id varchar(50) NOT NULL,"
    "  section_id varchar(100) NOT NULL,"
    "  start float(10),"
    "  duration float(10),"
    "  confidence float(10),"
    "  loudness float(10),"
    "  tempo float(10),"
    "  tempo_confidence float(10),"
    "  key_ float(10),"
    "  key_confidence float(10),"
    "  mode float(10),"
    "  mode_confidence float(10),"
    "  time_signature float(10),"
    "  time_signature_confidence float(10),"
    "  PRIMARY KEY (section_id)"
    ") ENGINE=InnoDB")

In [68]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table artists: OK
Creating table albums: OK
Creating table songs: OK
Creating table features: OK
Creating table sections: OK


## Connect to Spotfiy API to add information into database

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id='40f2cc0372324a4c85b28e189aa51f81', client_secret='d8dde2eeaa044974bfbda56e9cb76c7f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
## get artist URI from Spotify desktop app to query API
jayz_uri = 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'
kanye_uri = 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'
rolling_stones_uri = 'spotify:artist:22bE4uQ6baNwSHPVcDxLCe'
bruce_springsteen_uri = 'spotify:artist:3eqjTLE0HfPfh78zjh6TqT'
red_hot_chilli_peppers_uri = 'spotify:artist:0L8ExT028jH3ddEcZwqJJ5'
kendrick_uri = 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'
fleetwood_mac_uri = 'spotify:artist:08GQAI4eElDnROBrJRGE0X'

uris = [jayz_uri, kanye_uri, rolling_stones_uri, bruce_springsteen_uri, red_hot_chilli_peppers_uri, kendrick_uri, fleetwood_mac_uri]


In [10]:
# Get artist information from API

def get_artist(uri):
   
    artist = sp.artist(uri)
    artist_dict = {'artist_id' : artist['id'],
          'name' : artist['name'],
          'genre' : (', ').join(artist['genres']),
          'popularity' : artist['popularity']      
    }
    return artist_dict

    


In [11]:
# Prepare artist information for insert into SQL database

artists= []
for artist in uris:
    artists.append(get_artist(artist))
    
insert_tuples = [tuple(x.values()) for x in artists]

In [13]:
# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in artists[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.artists ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Use artist ids to query each artist's albums to insert into the database

In [64]:
stmt = """SELECT artist_id FROM music.artists"""
cursor.execute(stmt)

artist_ids = [x[0] for x in cursor.fetchall()]


In [82]:
artist_uris = ['spotify:artist:'+ artist_id for artist_id in artist_ids]
 


In [58]:
# hova_albums = sp.artist_albums(artist_id=jayz_uri, limit=45, country='US')['items']

def parse_album(album):

        album_dict = {'album_id' : album['id'],
                     'album_name' : album['name'],
                     }
        if len(album['artists']) > 3:
            album_dict['artist_id1'] = album['artists'][0]['id']
            album_dict['artist_id2'] = album['artists'][1]['id']
            album_dict['artist_id3'] = album['artists'][2]['id']
        elif len(album['artists']) > 1:
            album_dict['artist_id1'] = album['artists'][0]['id']
            album_dict['artist_id2'] = album['artists'][1]['id']
            album_dict['artist_id3'] = None
        else:
            album_dict['artist_id1'] = album['artists'][0]['id']
            album_dict['artist_id2'] = None
            album_dict['artist_id3'] = None
            
        album_dict['release_date']= album['release_date']
        album_dict['total_tracks'] = album['total_tracks']
        
        return album_dict


In [97]:
all_albums = []
for artist in artist_uris:
    albums = sp.artist_albums(artist_id=artist, limit=50, country='US')['items']
    all_albums.append(parse_album(albums[0]))
    for album in albums[1:]:
        if album['album_type'] != 'album':
            continue
        elif album['name'] not in [x['album_name'] for x in all_albums]:
            all_albums.append(parse_album(album))
        else:
            continue
    

            

In [103]:
for album in all_albums:
    if album['artist_id1'] == '0LyfQWJT6nXafLPZqxe9Of':
        all_albums.remove(album)
    else:
        continue
all_albums

[{'album_id': '3mlM7EnbGFBqcYNZ32ULWh',
  'album_name': 'Before the Beginning - 1968-1970 Rare Live & Demo Sessions (Remastered)',
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2019-11-15',
  'total_tracks': 41},
 {'album_id': '06JqOkwwy91OxrApXclzYf',
  'album_name': "50 Years - Don't Stop",
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2018-11-16',
  'total_tracks': 50},
 {'album_id': '6dMdadS6Ng71pD9As93ufx',
  'album_name': 'Say You Will',
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2003-04-15',
  'total_tracks': 18},
 {'album_id': '6YEnWIpSS19x0lKzJPi9Ej',
  'album_name': "Shrine '69",
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2000-04-28',
  'total_tracks': 10},
 {'album_id': '70MrKoi2AdCFBxKqeXDPHZ',
  'album_name': 'The Dance',
  'artist_id1

In [106]:
# Prepare artist information for insert into SQL database
    
insert_tuples = [tuple(x.values()) for x in all_albums]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in all_albums[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.albums ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Pull tracks from albums in database to insert track information for each song

In [109]:
stmt = """SELECT album_id FROM music.albums"""
cursor.execute(stmt)

album_ids = [x[0] for x in cursor.fetchall()]


In [121]:
## Parse songs from album tracks for the relevant information to be uploaded into the DB

def parse_songs(album_id):
    album_tracks = sp.album_tracks(album_id)
    songs = []
    for song in album_tracks['items']:
        song_dict = {'song_id' : song['id'],
                     'song_name' : song['name'],
                     }
        if len(song['artists']) > 3:
            song_dict['artist_id1'] = song['artists'][0]['id']
            song_dict['artist_id2'] = song['artists'][1]['id']
            song_dict['artist_id3'] = song['artists'][2]['id']
        elif len(song['artists']) > 1:
            song_dict['artist_id1'] = song['artists'][0]['id']
            song_dict['artist_id2'] = song['artists'][1]['id']
            song_dict['artist_id3'] = None
        else:
            song_dict['artist_id1'] = song['artists'][0]['id']
            song_dict['artist_id2'] = None
            song_dict['artist_id3'] = None
            
        song_dict['album_id']= album_id
        song_dict['track_number'] = song['track_number']
        
        if 'popularity' in song.keys():
            song_dict['popularity'] = song['popularity']
        else:
            song_dict['popularity'] = None
        
        songs.append(song_dict)
            
    return songs


In [133]:
all_songs = []

for album in album_ids:
    all_songs.append(parse_songs(album))

flat_songs = [item for album in all_songs for item in album]

In [135]:
# Prepare artist information for insert into SQL database
    
insert_tuples = [tuple(x.values()) for x in flat_songs]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in flat_songs[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.songs ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Use song ids to pull features for each song to upload into db

In [139]:
stmt = """SELECT song_id FROM music.songs"""
cursor.execute(stmt)

song_ids = [x[0] for x in cursor.fetchall()]


In [164]:


def parse_features(song):
    song_dict = sp.audio_features(song)[0]
    features = {'song_id' : song,
                'feature_id' : song_dict['id'] + str(song_dict['duration_ms']),
                'danceability' : song_dict['danceability'],
                'energy' : song_dict['energy'],
                'key_' : song_dict['key'],
                'loudness' : song_dict['loudness'],
                'mode' : song_dict['mode'],
                'speechiness' : song_dict['speechiness'],
                'acousticness' : song_dict['acousticness'],
                'instrumentalness' : song_dict['instrumentalness'],
                'liveness' : song_dict['liveness'],
                'valence' : song_dict['valence'],
                'tempo' : song_dict['tempo'],
                'duration_ms' : song_dict['duration_ms'],
                'time_signature' : song_dict['time_signature']
        
    }
    return features
    

In [166]:
song_features = []
for song in song_ids:
    song_features.append(parse_features(song))


retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [169]:
# Prepare artist information for insert into SQL database
    
insert_tuples = [tuple(x.values()) for x in song_features]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in song_features[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.features ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Use song ids to query sections from the spotify api to upload section information into the database

In [188]:
def get_sections(song):
    sections = sp.audio_analysis(song)['sections']
    sections_list = []
    for section in sections:
                        
                        sections_list.append(
                        {'song_id' : song,
                        'section_id' : song + str(section['start']),
                        'start' : section['start'],
                        'duration': section['duration'],
                        'confidence': section['confidence'],
                        'loudness' : section['loudness'],
                        'tempo' : section['tempo'],
                        'tempo_confidence' : section['tempo_confidence'],
                        'key_' : section['key'],
                        'key_confidence' : section['key_confidence'],
                        'mode' : section['mode'],
                        'mode_confidence' : section['mode_confidence'],
                        'time_signature' : section['time_signature'],
                        'time_signature_confidence' : section['time_signature_confidence']})
        
    return sections_list

In [ ]:
all_sections = []
for song in song_ids:
    all_sections.append(get_sections(song))

In [ ]:
flat_sections = [item for section in all_sections for item in section]

In [ ]:
insert_tuples = [tuple(x.values()) for x in flat_sections]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in flat_sections[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.sections ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

In [192]:
 sections = sp.audio_analysis(song_ids[2800])['sections']

In [193]:
sections

[{'confidence': 1.0,
  'duration': 5.81187,
  'key': 7,
  'key_confidence': 0.108,
  'loudness': -11.872,
  'mode': 1,
  'mode_confidence': 0.537,
  'start': 0.0,
  'tempo': 105.617,
  'tempo_confidence': 0.963,
  'time_signature': 4,
  'time_signature_confidence': 1.0},
 {'confidence': 0.763,
  'duration': 18.675,
  'key': 0,
  'key_confidence': 0.813,
  'loudness': -8.167,
  'mode': 1,
  'mode_confidence': 0.735,
  'start': 5.81187,
  'tempo': 105.991,
  'tempo_confidence': 0.583,
  'time_signature': 4,
  'time_signature_confidence': 1.0},
 {'confidence': 0.171,
  'duration': 9.03736,
  'key': 5,
  'key_confidence': 0.073,
  'loudness': -7.338,
  'mode': 1,
  'mode_confidence': 0.446,
  'start': 24.48687,
  'tempo': 106.38,
  'tempo_confidence': 0.591,
  'time_signature': 4,
  'time_signature_confidence': 1.0},
 {'confidence': 0.199,
  'duration': 12.45088,
  'key': 0,
  'key_confidence': 0.63,
  'loudness': -7.986,
  'mode': 1,
  'mode_confidence': 0.534,
  'start': 33.52422,
  'tem

In [163]:
sp.audio_features(song_ids[1])[0]

{'acousticness': 0.14,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/00hKh3kUrEgNSwPdfCmu2X',
 'danceability': 0.299,
 'duration_ms': 431573,
 'energy': 0.959,
 'id': '00hKh3kUrEgNSwPdfCmu2X',
 'instrumentalness': 0.122,
 'key': 6,
 'liveness': 0.935,
 'loudness': -3.51,
 'mode': 0,
 'speechiness': 0.119,
 'tempo': 126.374,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/00hKh3kUrEgNSwPdfCmu2X',
 'type': 'audio_features',
 'uri': 'spotify:track:00hKh3kUrEgNSwPdfCmu2X',
 'valence': 0.338}

In [22]:
### should engineer a feature for number of sessions
## engineer feature for variance in loadness of sectionsd
# https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

for key in sp.audio_analysis(gotta_have_it_uri)['sections'][0].keys():
    print (key + " float (10)")

start float (10)
duration float (10)
confidence float (10)
loudness float (10)
tempo float (10)
tempo_confidence float (10)
key float (10)
key_confidence float (10)
mode float (10)
mode_confidence float (10)
time_signature float (10)
time_signature_confidence float (10)


In [15]:
jayz_uri = 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'
wtt_uri = sp.artist_albums(artist_id=jayz_uri)['items'][11]['uri']

In [16]:
gotta_have_it_uri = sp.album_tracks(wtt_uri)['items'][4]['uri']

In [21]:
# https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/
sp.audio_features(gotta_have_it_uri)[0]


{'acousticness': 0.411,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6JFLZH9vCzgqnUToBdiHQb',
 'danceability': 0.74,
 'duration_ms': 140747,
 'energy': 0.574,
 'id': '6JFLZH9vCzgqnUToBdiHQb',
 'instrumentalness': 0,
 'key': 4,
 'liveness': 0.262,
 'loudness': -6.938,
 'mode': 1,
 'speechiness': 0.245,
 'tempo': 149.762,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/6JFLZH9vCzgqnUToBdiHQb',
 'type': 'audio_features',
 'uri': 'spotify:track:6JFLZH9vCzgqnUToBdiHQb',
 'valence': 0.33}

In [53]:
sp.artist(jayz_uri)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
 'followers': {'href': None, 'total': 4998282},
 'genres': ['east coast hip hop',
  'gangster rap',
  'hip hop',
  'pop rap',
  'rap',
  'southern hip hop'],
 'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
 'id': '3nFkdlSjzX9mRTtwJOzDYB',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/4912d27d6b01dd790313d8ef76586be6b100550f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/027c5605d895158497ff377d92d69fe89dba1730',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/1420226e471d0e56ae4de7cb28ba2eb4de0a29ce',
   'width': 160}],
 'name': 'JAY-Z',
 'popularity': 86,
 'type': 'artist',
 'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'}

In [54]:
sp.artist_albums(artist_id=jayz_uri)['items'][11]

{'album_group': 'album',
 'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
   'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
   'id': '3nFkdlSjzX9mRTtwJOzDYB',
   'name': 'JAY-Z',
   'type': 'artist',
   'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
   'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
   'id': '5K4W6rqBFWDnAN6FQUkS6x',
   'name': 'Kanye West',
   'type': 'artist',
   'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
 'available_markets': ['CA', 'JP', 'MX', 'US'],
 'external_urls': {'spotify': 'https://open.spotify.com/album/7mCeLbChyegbRwwKK5shJs'},
 'href': 'https://api.spotify.com/v1/albums/7mCeLbChyegbRwwKK5shJs',
 'id': '7mCeLbChyegbRwwKK5shJs',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/c0df37da32cbe3212fbc95fd6863e2e9047b086f',
   'widt

In [55]:
sp.track(gotta_have_it_uri)

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
    'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
    'id': '3nFkdlSjzX9mRTtwJOzDYB',
    'name': 'JAY-Z',
    'type': 'artist',
    'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
  'available_markets': ['CA', 'JP', 'MX', 'US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/7mCeLbChyegbRwwKK5shJs'},
  'href': 'https://api.spotify.com/v1/albums/7mCeLbChyegbRwwKK5shJs',
  'id': '7mCeLbChyegbRwwKK5shJs',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/c0df37da32cbe3212fbc95fd6863e2e9047b086f',
    '

In [ ]:
sp.tracks()